In [3]:
import pandas as pd
import numpy as np
import datetime as dt
import math
import matplotlib.pyplot as plt

from pandas.tseries.offsets import BDay, Day
from tqdm import tqdm

PATH = "C:/Users/jackl/OneDrive/Documents/finance_research/japan_qe/"

In [4]:
xls = pd.ExcelFile(PATH+'raw_data/bloomberg/topix_weights.xlsx')
topix_df = pd.read_excel(xls, 'Sheet1')
match_df = pd.read_excel(xls, 'Sheet2')

In [22]:
nikkei_df = pd.read_excel(PATH+'raw_data/bloomberg/nikkei_weights.xlsx')

In [23]:
nikkei_df.columns = nikkei_df.iloc[4]
nikkei_df = nikkei_df.iloc[7:]

In [24]:
columns_list = list(nikkei_df.columns)
columns_list[0] = 'date'
nikkei_df.columns = columns_list

In [25]:
nikkei_df['date'] = list(topix_df.iloc[5:]['Start Date'])

In [27]:
df_temp_list = []
for ticker in tqdm(columns_list[1:]): 
    df_temp = nikkei_df[['date', ticker]]
    df_temp.columns = ['date', 'weight']
    df_temp['ticker'] = [ticker for i in range(len(df_temp))]
    df_temp_list.append(df_temp)

  0%|                                                                                         | 0/2183 [00:00<?, ?it/s]<ipython-input-27-2ea178fe1714>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_temp['ticker'] = [ticker for i in range(len(df_temp))]
100%|█████████████████████████████████████████████████████████████████████████████| 2183/2183 [00:08<00:00, 260.74it/s]


In [28]:
nikkei_df_reo = pd.concat(df_temp_list, axis=0)
nikkei_df_reo['date'] = pd.to_datetime(nikkei_df_reo['date'])

In [15]:
match_df_reo = match_df.T.reset_index()
match_df_reo.columns = ['ticker', 'SEDOL', 'ISIN']
match_df_reo = match_df_reo.iloc[1:]

In [16]:
nikkei_df_reo = pd.merge(nikkei_df_reo, match_df_reo, on='ticker')

In [17]:
def clean(x): 
    try: 
        if float(x) > -99: 
            return x
    except: 
        return math.nan

In [18]:
nikkei_df_reo['weight'] = nikkei_df_reo['weight'].apply(lambda x: clean(x))

In [20]:
nikkei_df_reo.drop_duplicates(['SEDOL'])

,date,weight,ticker,SEDOL,ISIN
0,2010-11-30,NaN,1301 JT Equity,6498706,JP3257200000
131,2010-11-30,0.0999,1332 JT Equity,6640927,JP3718800000
262,2010-11-30,NaN,1333 JT Equity,BKGHM19,JP3876600002
393,2010-11-30,NaN,1352 JT Equity,6432823,JP3839400003
524,2010-11-30,NaN,1375 JT Equity,BMF52S5,JP3947010009
...,...,...,...,...,...
285318,2010-11-30,NaN,9991 JT Equity,6491794,JP3225500002
285449,2010-11-30,NaN,9993 JT Equity,6986814,JP3935800007
285580,2010-11-30,NaN,9994 JT Equity,6986803,JP3943800007
285711,2010-11-30,NaN,9995 JT Equity,6301042,JP3130600004


In [184]:
nikkei_df_reo.to_pickle('checkpoint_data/nikkei_weights_bloomberg.pkl')